<hr style="border:1px solid gray"> </hr>

# Overdetermined systems

A system of polynomial equations $f=(f_1(x_1,\ldots, x_m),\ldots,  f_n(x_1,\ldots,x_m))$ is called *overdetermined*, if it has more equations than variables; i.e., when $n>m$. HomotopyContinuation.jl can solve overdetermined systems. Here is a simple example.

$$f(x,y,z) = \begin{bmatrix} xz-y^2 \\\ y-z^2 \\\ x-yz \\\ x + y + z + 1\end{bmatrix}.$$

This system has 4 equation in 3 variables. One might expect that it has no solution, but actually it has solutions, as is explained [here](https://en.wikipedia.org/wiki/Rational_normal_curve).

The Julia code is as follows

In [1]:
using HomotopyContinuation
@var x y z
f = [
    x*z - y^2, 
    y - z^2, 
    x - y*z, 
    x + y + z + 1
]

solve(f)

Tracking 5 paths... 100%|███████████████████████████████| Time: 0:00:08
  # paths tracked:                  5
  # non-singular solutions (real):  3 (1)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         3 (1)


Result with 3 solutions
• 5 paths tracked
• 3 non-singular solutions (1 real)
• 2 excess solutions
• random_seed: 0x09097a52
• start_system: :polyhedral


Internally, HomotopyContinuation multiplies $f$ with a random $3\times 4$ matrix $A$ and then solves the quadratic system $Af$. For a generic $A$ the isolated zeros of $f$ are isolated zeros of $Af$. We can do this by hand:

In [2]:
A = randn(ComplexF64, 3, 4)
S = solve(A*f)

Tracking 6 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  6
  # non-singular solutions (real):  5 (1)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         5 (1)


Result with 5 solutions
• 6 paths tracked
• 5 non-singular solutions (1 real)
• random_seed: 0x1c225a19
• start_system: :polyhedral


and sort out the zeros of $f$ by hand by first evaluation $f$ at the entries of `S`

In [3]:
f_values = map(s -> [fi([x, y, z] => s) for fi in f], solutions(S))

5-element Array{Array{Complex{Float64},1},1}:
 [-0.012158421332225916 - 0.10406022887657514im, 0.3967818617169086 - 0.03225032557197248im, -0.32874183818540786 - 0.48373817985195605im, 1.0649512911632886 - 0.12264532784621052im]
 [-2.220446049250313e-16 - 1.2854157372834284e-16im, 2.220446049250313e-16 - 2.3232446801547724e-16im, 1.520267170027599e-16 + 4.440892098500626e-16im, 3.984190017847113e-17 - 7.771561172376096e-16im]
 [-2.220446049250313e-16 - 5.927276076647478e-17im, 0.0 + 1.0142380740177176e-16im, 2.220446049250313e-16 + 1.4389058481998623e-16im, -2.220446049250313e-16 - 2.870804064647678e-16im]
 [-2.220446049250313e-16 - 2.8970810878801883e-16im, 2.220446049250313e-16 - 7.095843253537515e-17im, 2.29577964005159e-16 - 4.440892098500626e-16im, -6.299789632200459e-16 - 8.881784197001252e-16im]
 [0.11694769383670156 - 0.3198258313947999im, 1.1988542569725888 + 0.4868758430996216im, -0.24648549288642396 - 1.8850087298907705im, 3.2704845012492303 + 1.2020190949217011im]

and then sorting out the zero entries

In [4]:
zero_entries = findall(v -> maximum(abs.(v)) < 1e-10, f_values)
println("\n and the zeros are at $zero_entries")


 and the zeros are at [2, 3, 4]


However, evaluation of $f$ at the solutions of $Af$ is not a stable way of finding the zeros of $f$. This is because $f$ and $tf$ have the same zeros for all $t\in\mathbb{C}\backslash\{0\}$. HomotopyContinuation.jl checks which zeros of $Af$ converge quadratically fast towards zeros of $f$ in order to make a decision.

<hr style="border:1px solid gray"> </hr>

# Composite systems

If your polynomial system $f$ is given as the composition of systems it is more efficient to *not* expand $f$, but exploit this structure! 

For instance, if

$$ f = \begin{bmatrix} ab - 2\\  ac- 1\end{bmatrix}, \quad\text{and}\quad g =  \begin{bmatrix}x + y\\ y + 3\\ x + 2\end{bmatrix},$$

then you solve $f\circ g$ by

In [6]:
using HomotopyContinuation
@var a b c x y
f = System([a * b - 2, a * c- 1])
g = System([x + y, y + 3, x + 2])

solve(f ∘ g)

Tracking 4 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  4
  # non-singular solutions (real):  2 (2)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (2)


Result with 2 solutions
• 4 paths tracked
• 2 non-singular solutions (2 real)
• random_seed: 0x2e4fc417
• start_system: :polyhedral


It is also possible to iterate the above process:

In [7]:
@var u v
h = System([u^2 - 1, u + v - 2])
solve(f ∘ g ∘ h)

Tracking 8 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  8
  # non-singular solutions (real):  4 (2)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         4 (2)


Result with 4 solutions
• 8 paths tracked
• 4 non-singular solutions (2 real)
• random_seed: 0x9dbfe33e
• start_system: :polyhedral
